In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('get_follower_data.csv')

In [3]:
df.head()

,Unnamed: 0,hours,mins,secs,text
0,0,16,30,36,How to UNPIVOT multiple columns into tidy pair...
1,1,16,0,4,"Shortest Path Algorithms, Part 2: Floyd�Warsha..."
2,2,15,8,1,Code to run SQL queries 10 times faster than B...
3,3,14,41,43,New to Data Visualization? Start with New York...
4,4,4,57,47,I wrote a Colab notebook that introduces diffe...


In [7]:
df['text'] = df['text'].apply(lambda x: x[0:-1].replace('\n\n', ' '))

In [8]:
import re

def clear_emoji(text):
    emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [9]:
df['text']=df['text'].apply(clear_emoji)

In [10]:
import numpy as np
import gensim

In [11]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore



In [12]:
import spacy

In [14]:

from spacy.tokenizer import Tokenizer

In [15]:
nlp = spacy.load("en_core_web_lg")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [43]:

STOP_WORDS = nlp.Defaults.stop_words.union(['&amp;',"don't","i'm","i've","it's","&gt;","i’m"])


In [44]:
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(df['text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        if (token.text.lower() not in STOP_WORDS) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [45]:
id2word = corpora.Dictionary(df['tokens'])

In [46]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [47]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=8
                  )

In [48]:
lda.print_topics()

[(0,
  '0.009*"new" + 0.004*"time" + 0.004*"=" + 0.003*"de" + 0.003*"data" + 0.003*"people" + 0.002*"learning" + 0.002*"y" + 0.002*"today." + 0.002*"que"'),
 (1,
  '0.004*"got" + 0.003*"que" + 0.003*"la" + 0.002*"need" + 0.002*"love" + 0.002*"un" + 0.002*"data" + 0.002*"de" + 0.002*"people" + 0.002*"want"'),
 (2,
  '0.004*"like" + 0.003*"la" + 0.002*"de" + 0.002*"con" + 0.002*"en" + 0.002*"lot" + 0.002*"join" + 0.002*"mi" + 0.002*"i…" + 0.002*"want"'),
 (3,
  '0.002*"new" + 0.002*"people" + 0.002*"world" + 0.002*"time" + 0.002*"need" + 0.002*"right" + 0.002*"data" + 0.002*"code" + 0.002*"un" + 0.001*"learning"'),
 (4,
  '0.005*"data" + 0.003*"working" + 0.003*"people" + 0.002*"today" + 0.002*"learning" + 0.002*"science" + 0.002*"use" + 0.002*"know" + 0.002*"best" + 0.002*"way"'),
 (5,
  '0.006*"de" + 0.003*"like" + 0.003*"great" + 0.003*"que" + 0.002*"best" + 0.002*"new" + 0.002*"sure" + 0.002*"las" + 0.002*"we…" + 0.002*"5"'),
 (6,
  '0.018*"de" + 0.014*"la" + 0.010*"el" + 0.009*"en" 

In [49]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [50]:

topics = [' '.join(t[0:5]) for t in words]

In [51]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
new time = de data

------ Topic 1 ------
got que la need love

------ Topic 2 ------
like la de con en

------ Topic 3 ------
new people world time need

------ Topic 4 ------
data working people today learning

------ Topic 5 ------
de like great que best

------ Topic 6 ------
de la el en que

------ Topic 7 ------
que learning data got day

------ Topic 8 ------
want data like r covid-19

------ Topic 9 ------
data new la good de

------ Topic 10 ------
end covid-19 gonna #machinelearning #ai

------ Topic 11 ------
new data people ⦁ ⠀

------ Topic 12 ------
people time free que it’s

------ Topic 13 ------
y work ibm que |

------ Topic 14 ------
people think learning like code

